# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [10]:
import pandas as pd

## 1 - Fase de Preparação dos Dados
---

Carregando o arquivo original de classes.

In [11]:
classes_original = pd.read_excel("dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa.
Essa classes possuem nível 2 na tabela original.

In [12]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
1,33809814,Temas Exclusivos de Pronunciamentos
2,33809634,Meio Ambiente
3,33809514,"Soberania, Defesa Nacional e Ordem Pública"
4,33808912,Política Social
5,33805362,Jurídico
6,33805317,Honorífico
7,33805137,Infraestrutura
8,33769167,Economia e Desenvolvimento
9,33768972,Organização do Estado
10,33685789,Administração Pública


Carrega o dataset contendo as leis que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [13]:
leis_classificadas_original = pd.read_excel("dados/ClassificacaoDeLeisOrdinariasDesde1900.xlsx", "Select mvw_u03_prc_doc_tema")
leis_classificadas_original["DES_CLASSE_RAIZ"] = leis_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
leis_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36156374,540837,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36158248,540987,33805842,Crédito Extraordinário,Classificação Temática Unificada / Orçamento P...,Orçamento Público
2,3,36158249,540998,33805842,Crédito Extraordinário,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36156375,541008,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
4,5,36156376,541015,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as leis (classificadas e não classificadas) e aplica as limpezas iniciais.

In [14]:
leis_original = pd.read_excel("dados/LeisOrdinariasDesde1900.xlsx", "Select mvw_s01_documento")
leis_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
leis_original.drop(columns="   ", inplace=True)
leis_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA
0,542244,Lei nº 689 de 20/09/1900,LEI-689-1900-09-20,Autoriza o Governo a recolher em conta corrent...
1,542305,Lei nº 741 de 26/12/1900,LEI-741-1900-12-26,Orça a receita geral da Republica dos Estados ...
2,542241,Lei nº 687 de 14/09/1900,LEI-687-1900-09-14,Fixa as Forças de terra para o exercicio de 1901
3,542412,Lei nº 834 de 30/12/1901,LEI-834-1901-12-30,Fixa a despeza geral da Republica dos Estados ...
4,542387,Lei nº 813 de 23/12/1901,LEI-813-1901-12-23,Fixa a receita geral da Republica dos Estados ...


Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas leis do dataset de leis classificadas.

In [15]:
leis = leis_original.merge(leis_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO","DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
leis = leis.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
leis.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
leis.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE
0,542244,Lei nº 689 de 20/09/1900,LEI-689-1900-09-20,Autoriza o Governo a recolher em conta corrent...,NaN,NaN
1,542305,Lei nº 741 de 26/12/1900,LEI-741-1900-12-26,Orça a receita geral da Republica dos Estados ...,NaN,NaN
2,542241,Lei nº 687 de 14/09/1900,LEI-687-1900-09-14,Fixa as Forças de terra para o exercicio de 1901,NaN,NaN
3,542412,Lei nº 834 de 30/12/1901,LEI-834-1901-12-30,Fixa a despeza geral da Republica dos Estados ...,NaN,NaN
4,542387,Lei nº 813 de 23/12/1901,LEI-813-1901-12-23,Fixa a receita geral da Republica dos Estados ...,NaN,NaN


Limpa os registros duplicados de leis que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [16]:
leis.drop_duplicates(inplace=True)
temp = leis[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
8844,549877,33805137.0,Infraestrutura
8845,549877,33769167.0,Economia e Desenvolvimento
10839,551926,33805362.0,Jurídico
10840,551926,33808912.0,Política Social
11031,552026,33769167.0,Economia e Desenvolvimento
...,...,...,...
15645,35847286,33805362.0,Jurídico
15648,35957120,33805137.0,Infraestrutura
15649,35957120,33769167.0,Economia e Desenvolvimento
15650,36198674,33769167.0,Economia e Desenvolvimento


In [19]:
leis_classificadas = leis.query("not COD_CLASSE.isnull()")
leis_classificadas.shape[0]

5668

In [20]:
leis_nao_classificadas = leis.query("COD_CLASSE.isnull()")
leis_nao_classificadas.shape[0]

9758